In [ ]:
import numpy as np

from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Library.HardwareModels import hardware_models_Quantinuum
from LogicalQ.NoiseModel import construct_noise_model_from_hardware_model
from LogicalQ.Optimizers import compute_effective_threshold, compute_gadget_costs, compute_constraint_model
from LogicalQ.Benchmarks import mirror_benchmarking
from LogicalQ.Experiments import execute_circuits, qec_cycle_efficiency_experiment
from LogicalQ.Analysis import qec_cycle_efficiency_scatter

## Compute the effective threshold

In [ ]:
_, _, effective_threshold = compute_effective_threshold(
    hardware_model=None,
    # hardware_model=hardware_models_Quantinuum["H1-1"],
    # hardware_model=hardware_models_Quantinuum["H2-1"],
    **steane_code,
    min_theta=0, max_theta=np.pi/4, n_theta=1,
    min_phi=0, max_phi=np.pi*2, n_phi=1,
    max_n_qec_cycles=1
)

## Compute gadget costs

In [ ]:
gadgets_library, gadget_infidelities, gadget_costs = compute_gadget_costs(
    gadgets_library=None,
    backend="aer_simulator", hardware_model=hardware_models_Quantinuum["H2-1"]
)

## Construct constraint model

In [ ]:
for n_qubits, n_qubit_noise_data in hardware_models_Quantinuum["H2-1"]["noise_params"].items():
    for noise_param_key, noise_param_data in n_qubit_noise_data.items():
        if noise_param_key in ["depolarizing_error", "amplitude_damping_error"]:
            for param_n_qubits, n_qubit_error_data in noise_param_data.items():
                for gate, gate_error_value in n_qubit_error_data.items():
                    print(gate, gate_error_value)

In [ ]:
compute_constraint_model(
    hardware_models_Quantinuum["H2-1"],
    **steane_code,
    optimizer=True,
    effective_threshold=0.03,
    gadget_costs=gadget_costs,
    constraint_model=None,
)

## Test constraint model

### Prepare experiment inputs

In [ ]:
def logical_circuit_factory(n_qubits, circuit_length):
    qc = mirror_benchmarking(n_qubits=n_qubits, circuit_length=circuit_length, seed=1111)
    return LogicalCircuit.from_physical_circuit(qc, **steane_code)

In [ ]:
log_data_mp = circuit_scaling_experiment(
    logical_circuit_factory,
    noise_model_factory,
    min_n_qubits=1,
    max_n_qubits=1,
    min_circuit_length=1,
    max_circuit_length=32,
    shots=1E2,
    with_mp=True
)

In [ ]:
data_list = []
for constraint_scan_key in constraint_scan_key_list:
    data = qec_cycle_efficiency_experiment(
        circuit_input=qc,
        qecc=steane_code,
        constraint_scan_keys=["effective_threshold", constraint_scan_key], constraint_scan_val_lists=[[effective_threshold], constraint_scan_values],
        backend="aer_simulator",
        hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None,
        noise_model=None,
        method="statevector", shots=1E3, with_mp=False,
        save_dir=None, save_filename=None
    )
    data_list.append(data)

In [ ]:
for data, constraint_scan_key in zip(data_list, constraint_scan_key_list):
    qec_cycle_efficiency_scatter(data, scan_keys=[constraint_scan_key])

In [ ]:
qci_list = [(list(datum["constraint_model"].values())[1], datum["qec_cycle_indices"]) for datum in data_list[0][0]["results"]]
print(qci_list)

In [ ]:
lqc = data_list[0][0]["logical_circuit"]
lqc.insert_qec_cycles(qec_cycle_indices=qci_list[-1][1])
lqc.draw("mpl")